In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import numpy as np
import os
x=[]
y=[]
c=0
for file in os.listdir('/kaggle/input/contrails-images-ash-color/contrails'):
    a=np.load('/kaggle/input/contrails-images-ash-color/contrails/'+file,mmap_mode='r')
    # Loading in 'r' format 
    xa = a[:,:,:3]
    ya = a[:,:,3]
    x.append(xa)
    y.append(ya)
    c+=1
    if(c%10==0):
        print(c)
        
    
        

10


In [7]:
import tensorflow as tf
tf_dfx = tf.data.Dataset.from_tensor_slices(x)
print("c1")
tf_dfx = tf_dfx.shuffle(200)
print("c2")
tf_dfy = tf.data.Dataset.from_tensor_slices(y)
print("c3")
tf_dfy = tf_dfy.shuffle(200)
print("c4")

#Here it takes much time 

c1
c2
c3
c4


In [10]:
train_size = int(len(tf_dfx) * 0.8)
xtrain = tf_dfx.take(train_size)
xtest = tf_dfx.skip(train_size)
ytrain = tf_dfy.take(train_size)
ytest = tf_dfy.skip(train_size)
print(len(xtrain))
print(len(xtest))

8
2


In [23]:
'''
xtrain = np.concatenate(list(xtrain.as_numpy_iterator()))
print(xtrain.shape)
xtrain = xtrain.reshape(8,256,256,3)
print(xtrain.shape)
#print(xtrain)
ytrain = np.concatenate(list(ytrain.as_numpy_iterator()))
print(ytrain.shape)
ytrain = ytrain.reshape(8,256,256,1)
print(ytrain.shape)
#print(ytrain)'''
xtest = np.concatenate(list(xtest.as_numpy_iterator()))
print(xtest.shape)
xtest = xtest.reshape(2,256,256,3)
print(xtest.shape)
ytest = np.concatenate(list(ytest.as_numpy_iterator()))
print(ytest.shape)
ytest = ytest.reshape(2,256,256,1)
print(ytest.shape)

#Use a function for this 

(512, 256, 3)
(2, 256, 256, 3)
(512, 256)
(2, 256, 256, 1)


In [21]:

from keras.models import *
from keras.layers import *
import numpy as np
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input, Cropping2D, Flatten
from keras.models import Model
from tensorflow import keras
def conv(inputs, num_filters):
    x = Conv2D(num_filters,(3,3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(inputs)
    #x = Dropout(0.1)(x)
    x = Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(x)
    return x
def deconv(inputs, conv_prev, num_filters):
    up = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    concat =  tf.keras.layers.concatenate([up, conv_prev])
    conva = conv(concat, num_filters)
    return conva
inputs = Input((256, 256, 3))
c1=conv(inputs,64)
p1=MaxPooling2D((2,2))(c1)
c2=conv(p1,128)
p2=MaxPooling2D((2,2))(c2)
c3=conv(p2,256)
p3=MaxPooling2D((2,2))(c3)
c4=conv(p3,512)
p4=MaxPooling2D((2,2))(c4)
c5=conv(p4,1024)

c6 = deconv(c5, c4, 512)
c7 = deconv(c6, c3, 256)
c8 = deconv(c7, c2, 128)
c9 = deconv(c8, c1, 64)
op= Conv2D(1,(1,1), activation='sigmoid',padding='same')(c9)
model = tf.keras.Model(inputs=[inputs], outputs=[op])
model.compile(optimizer='adam', loss=dice_loss, metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_19[0][0]']              
                                )                                                           

In [20]:

from keras import backend as K
def dice_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    total_pixels = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    dice_coefficient = (2. * intersection + smooth) / (total_pixels + smooth)
    return 1. - dice_coefficient

In [22]:
results = model.fit(xtrain,ytrain,batch_size=2,epochs=2)

Epoch 1/2
4/4 [==============================] - 37s 8s/step - loss: 0.9836 - accuracy: 0.0256
Epoch 2/2
4/4 [==============================] - 31s 8s/step - loss: 0.9833 - accuracy: 0.0091


In [26]:
import cv2
test_img = xtest[1]
#test_img = cv2.cvtColor(test_img,cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img,axis=0)
print(test_img.shape)
prediction=model.predict(test_img)
print(prediction.shape)
prediction = prediction.reshape(256,256,1)
print(ytest[1].shape)


(1, 256, 256, 3)
1/1 [==============================] - 1s 1s/step
(1, 256, 256, 1)
(256, 256, 1)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(prediction)
print(np.sum(prediction))
#predicted image

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(ytest[1])
print(np.sum(ytest))
#Actual image 